# Debugging

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Q: "No debugger for your code. What do you think?"**

**A: "I would NOT be able to code!"**

- Who does "print-line-debugging"?
- Who likes debugging in tensorflow?
- What is the intersection of those two groups?


## IPDB cheatsheet
IPython Debugger

Taken from http://wangchuan.github.io/coding/2017/07/12/ipdb-cheat-sheet.html

- h(help): Print help

- n(ext): Continue execution until the next line in the current function is reached or it returns.
- s(tep): Execute the current line, stop at the first possible occasion (either in a function that is called or in the current function).
- r(eturn): Continue execution until the current function returns.
- c(ont(inue)): Continue execution, only stop when a breakpoint is encountered.

- r(eturn): Continue execution until the current function returns.
- a(rgs): Print the argument list of the current function.

Note: Python 3.7 has `breakpoint()` built-in! [[PEP 553]](https://www.python.org/dev/peps/pep-0553/)

In [2]:
from IPython.core.debugger import set_trace

In [3]:
def my_function(x):
    answer = 42
    set_trace()
    answer += x
    return answer

my_function(12)

> <ipython-input-3-0515935c5419>(4)my_function()
      2     answer = 42
      3     set_trace()
----> 4     answer += x
      5     return answer
      6 

ipdb> n
> <ipython-input-3-0515935c5419>(5)my_function()
      3     set_trace()
      4     answer += x
----> 5     return answer
      6 
      7 my_function(12)

ipdb> n
--Return--
54
> <ipython-input-3-0515935c5419>(5)my_function()
      3     set_trace()
      4     answer += x
----> 5     return answer
      6 
      7 my_function(12)

ipdb> c


54

## Example: debuging a NN

In [4]:
X = torch.rand((5, 3))
X

tensor([[0.2264, 0.5846, 0.5872],
        [0.9644, 0.2515, 0.6260],
        [0.3210, 0.4044, 0.1913],
        [0.6440, 0.4672, 0.3134],
        [0.7453, 0.9080, 0.4492]])

In [5]:
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(3, 1)
    
    def forward(self, X):
        # set_trace()
        x = self.lin(X)
        return X

    
model = MyModule()
y_ = model(X)

assert y_.shape == (5, 1), y_.shape

AssertionError: torch.Size([5, 3])

## Debug Layer

In [6]:
class DebugModule(nn.Module):
    def forward(self, x):
        set_trace()
        return x

In [7]:
model = nn.Sequential(
    nn.Linear(1, 5),
    DebugModule(),
    nn.Linear(5, 1),
)

In [8]:
X = torch.unsqueeze(torch.tensor([1.]), dim=0)
model(X)

> <ipython-input-6-a95e71519cda>(4)forward()
      1 class DebugModule(nn.Module):
      2     def forward(self, x):
      3         set_trace()
----> 4         return x

ipdb> print(x)
tensor([[-0.0019,  0.0473,  1.7195,  0.9555,  0.1795]],
       grad_fn=<AddmmBackward>)
ipdb> c


tensor([[0.2864]], grad_fn=<AddmmBackward>)

## Tensorboard and `tensorboardX`
Tensorboard and `tensorboardX` are also great to debug a model, e.g. to look at the gradients.